# Projeto Marinha do Brasil

## Laboratório de Processamento de Sinais - UFRJ

### Autor: Vinícius dos Santos Mello <viniciusdsmello@poli.ufrj.br>

In [1]:
import os
import sys
import time
from datetime import datetime, timedelta
sys.path.insert(0,'..')

from noveltyDetectionConfig import CONFIG
import numpy as np
import itertools
import multiprocessing
import pprint 

from SAENoveltyDetectionAnalysis import SAENoveltyDetectionAnalysis

num_processes = multiprocessing.cpu_count()

from Functions.telegrambot import Bot

my_bot = Bot("lisa_thebot")

# Enviroment variables
data_path = CONFIG['OUTPUTDATAPATH']
results_path = CONFIG['PACKAGE_NAME']

training_params = {
    "Technique": "StackedAutoEncoder",
    "TechniqueParameters": {
        "allow_change_weights": False #False
    },
    "DevelopmentMode": False,
    "DevelopmentEvents": 400,
    "NoveltyDetection": True,
    "InputDataConfig": {
        "database": "4classes",
        "n_pts_fft": 1024,
        "decimation_rate": 3,
        "spectrum_bins_left": 400,
        "n_windows": 1,
        "balance_data": True
    },
    "OptmizerAlgorithm": {
        "name": "Adam",
        "parameters": {
            "learning_rate": 0.001,
            "beta_1": 0.90,
            "beta_2": 0.999,
            "epsilon": 1e-08,
            "learning_decay": 1e-6,
            "momentum": 0.3,
            "nesterov": True
        }
    },
    "HyperParameters": {
        "n_folds": 4,#10, #4,
        #"n_epochs": 300,#500,
        "pretraining_n_epochs": 300,
        "finetuning_n_epochs": 300,
        "n_inits": 1,#1, #2,
        "batch_size": 128,#128, #256,
        "kernel_initializer": "uniform",
        "bias_initializer": "ones", #zeros
        "encoder_activation_function": "sigmoid",
        "decoder_activation_function": "sigmoid",
        "classifier_output_activation_function": "sigmoid",
        "norm": "mapstd",
        "metrics": ["accuracy"],
        "loss": "mean_squared_error",
        "dropout": False,
        "dropout_parameter": 0.00,
        "regularization": None, #None
        "regularization_parameter": 0.00 #0.00
    },
    "callbacks": {
        "EarlyStopping": {
            "patience": 30,
            "monitor": "val_loss"
        }
    }
}
analysis = SAENoveltyDetectionAnalysis(parameters=training_params,
                                       model_hash="",
                                       load_hash=False, load_data=True, verbose=True)
all_data, all_trgt, trgt_sparse = analysis.getData()

SAE = analysis.createSAEModels()

trn_data = analysis.trn_data
trn_trgt = analysis.trn_trgt
trn_trgt_sparse = analysis.trn_trgt_sparse

Using TensorFlow backend.


[+] Time to read data file: 1.05872392654 seconds
Qtd event of A is 12939
Qtd event of B is 29352
Qtd event of C is 11510
Qtd event of D is 23760

Biggest class is B with 29352 events
Total of events in the dataset is 77561
Balacing data...
DataHandler Class: CreateEventsForClass
Original Size: (12939, 400)
DataHandler Class: CreateEventsForClass
Original Size: (29352, 400)
DataHandler Class: CreateEventsForClass
Original Size: (11510, 400)
DataHandler Class: CreateEventsForClass
Original Size: (23760, 400)
Reading from /home/vinicius.mello/Workspace/SonarAnalysis_2/Results/NoveltyDetection/4_folds_cross_validation_balanced_data.jbl


In [2]:
pp = pprint.PrettyPrinter(indent=1)
print analysis.model_hash
print analysis.getBaseResultsPath()
pp.pprint(analysis.parameters)

e0778959d0a01951aac45a34a4a6055ec2bb3f5a484d83c63458c65d59d93257
/home/vinicius.mello/Workspace/SonarAnalysis_2/Results/NoveltyDetection/StackedAutoEncoder/outputs/e0778959d0a01951aac45a34a4a6055ec2bb3f5a484d83c63458c65d59d93257
{'DevelopmentEvents': 400,
 'DevelopmentMode': False,
 'HyperParameters': {'batch_size': 128,
                     'bias_initializer': 'ones',
                     'classifier_output_activation_function': 'sigmoid',
                     'decoder_activation_function': 'sigmoid',
                     'dropout': False,
                     'dropout_parameter': 0.0,
                     'encoder_activation_function': 'sigmoid',
                     'finetuning_n_epochs': 300,
                     'kernel_initializer': 'uniform',
                     'loss': 'mean_squared_error',
                     'metrics': ['accuracy'],
                     'n_folds': 4,
                     'n_inits': 4,
                     'norm': 'mapstd',
                     'pretraining_

### Perform the training of the model

In [ ]:
for inovelty in range(len(analysis.class_labels)):
    startTime = time.time()
    
    data=trn_data[inovelty]
    trgt=trn_trgt[inovelty]
    trainingType = "neuronSweep"
    analysis.train(layer=1,
                   inovelty=inovelty,
                   fineTuning=True,
                   trainingType=trainingType, #foldSweep, neuronSweep, normal, layerSweep
                   hidden_neurons=[400,350,300,250,200,150,100,50],
                   neurons_variation_step=50,
                   numThreads=8,
                   model_hash=analysis.model_hash)
    
    duration = str(timedelta(seconds=float(time.time() - startTime)))
    print "The training of the model for novelty class {0} took {1} to be performed\n".format(analysis.class_labels[inovelty], duration)

python sae_train.py --layer 1 --novelty 0 --finetunning 1 --threads 8 --type neuronSweep --hiddenNeurons 400x350x300x250x200x150x100x50 --neuronsVariationStep 50 --modelhash 73b6bb9fc265bc775379a0d422a1a35f487edbfb37c2ff99a77f9e7a1a4dd1f5


### Grid Search

In [2]:
import copy
num_values = 4
gridSearch = [copy.deepcopy(training_params) for i in range(num_values)]
for params in gridSearch:
    params['HyperParameters']['regularization_parameter'] = 2**-i
    print params['HyperParameters']['regularization_parameter']
    i = i - 1
    
gridSearch[3]

0.125
0.25
0.5
1


{'DevelopmentEvents': 400,
 'DevelopmentMode': False,
 'HyperParameters': {'batch_size': 128,
  'bias_initializer': 'ones',
  'classifier_output_activation_function': 'sigmoid',
  'decoder_activation_function': 'sigmoid',
  'dropout': False,
  'dropout_parameter': 0.0,
  'encoder_activation_function': 'sigmoid',
  'kernel_initializer': 'uniform',
  'loss': 'mean_squared_error',
  'metrics': ['accuracy'],
  'n_epochs': 50,
  'n_folds': 4,
  'n_inits': 1,
  'norm': 'mapminmax',
  'regularization': None,
  'regularization_parameter': 1},
 'InputDataConfig': {'balance_data': True,
  'database': '4classes',
  'decimation_rate': 3,
  'n_pts_fft': 1024,
  'n_windows': 1,
  'spectrum_bins_left': 400},
 'NoveltyDetection': True,
 'OptmizerAlgorithm': {'name': 'Adam',
  'parameters': {'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-08,
   'learning_decay': 1e-06,
   'learning_rate': 0.001,
   'momentum': 0.3,
   'nesterov': True}},
 'Technique': 'StackedAutoEncoder',
 'TechniqueParameters': 